In [11]:
import os
path = 'D:/python_project/hyunmin_project/운동동작분류/'

In [12]:
import glob
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pylab as plt
from tqdm import tqdm
import re 
import tensorflow as tf
import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras_self_attention import SeqSelfAttention
import random 
import time
from scipy import fftpack

train=pd.read_csv(path + 'open/train_features.csv')
train_label=pd.read_csv(path + 'open/train_labels.csv')
test=pd.read_csv(path + 'open/test_features.csv')
submission=pd.read_csv(path + 'open/sample_submission.csv')

In [13]:
# 26번을 제외한 id 리스트
feature = list(train_label[train_label['label'] != 26]['id'])

In [6]:
# train 데이터에서 26번을 삭제시킨다.
temp = []
for n in tqdm(range(train.shape[0])):
    if train['id'][n] in feature:
        temp.append(train.iloc[n])

100%|███████████████████████████████████████████████████████████████████| 1875000/1875000 [09:04<00:00, 3446.01it/s]


In [7]:
# 26번을 삭제시킨 데이터프레임
without = pd.DataFrame(data=np.array(temp), columns=train.columns)
without = without.astype({'id':int, 'time':int})
without

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
964195,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
964196,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
964197,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
964198,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [6]:
without.to_csv('./without.csv')

NameError: name 'without' is not defined

In [14]:
without = pd.read_csv('./without.csv', index_col=0)
without

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
964195,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
964196,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
964197,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
964198,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [15]:
# https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# 데이터 증강을 통해 과접합을 줄여보자 - DACON
# 증강할 데이터 정리

without_train = []

for uid in tqdm(without['id'].unique()):
    temp = np.array(without[without['id'] == uid].iloc[:,2:], np.float32).T
    without_train.append(temp)

without_train = np.array(without_train, np.float32)
without_train = without_train[:,:,:,np.newaxis].reshape(-1,600,6)

without_label = train_label[train_label['label'] != 26]['label']

100%|██████████████████████████████████████████████████████████████████████████| 1607/1607 [00:02<00:00, 589.35it/s]


In [16]:
# https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# 데이터 증강을 통해 과접합을 줄여보자 - DACON
# 원본 데이터 정리
import tensorflow as tf

x_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis].reshape(-1,600,6)

y_train = train_label['label']
y_categorical = tf.keras.utils.to_categorical(y_train)

x_test = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:,np.newaxis].reshape(-1,600,6)

100%|████████████████████████████████████████████████████████████████████████████| 782/782 [00:00<00:00, 917.06it/s]


In [17]:
def aug(data, shift):
    shift_data = np.roll(data, shift, axis=2)
    return shift_data

# 데이터 증강
shift_data = []
shift_label = []
for n in tqdm(range(5)):
    shifted = aug(without_train, int(random.random() * 1000))
    shift_data.append(shifted)
    shift_label.append(without_label)

for n in tqdm(range(5)):
    shifted = aug(without_train, int(random.random() * 900))
    shift_data.append(shifted)
    shift_label.append(without_label)

for n in tqdm(range(5)):
    shifted = aug(without_train, int(random.random() * 700))
    shift_data.append(shifted)
    shift_label.append(without_label)
    
for n in tqdm(range(5)):
    shifted = aug(without_train, int(random.random() * 500))
    shift_data.append(shifted)
    shift_label.append(without_label)
    
for n in tqdm(range(5)):
    shifted = aug(without_train, int(random.random() * 300))
    shift_data.append(shifted)
    shift_label.append(without_label)
    
for n in tqdm(range(5)):
    shifted = aug(without_train, int(n * 100))
    shift_data.append(shifted)
    shift_label.append(without_label)
    

shift_data = np.array(shift_data).reshape(-1,600,6)
shift_label = np.array(shift_label).reshape(-1,1)
shift_categorical = tf.keras.utils.to_categorical(shift_label)

100%|█████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 65.11it/s]


In [18]:
# 원본 데이터와 증강 데이터 합치기
concat_train = np.concatenate((x_train, shift_data), axis=0)
concat_label = np.concatenate((y_categorical, shift_categorical), axis=0)
print(concat_train.shape)
print(concat_label.shape)

(51335, 600, 6)
(51335, 61)


In [23]:
concat_train = pd.DataFrame(np.concatenate(concat_train))

concat_train.columns = ['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
concat_train.head()

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,1.206087,1.287696,1.304609,1.293095,1.300887,1.289304
1,1.281405,1.242273,1.198871,1.113677,1.192241,1.230094
2,1.175210,1.170251,1.075277,0.993865,0.956588,0.977357
3,0.997987,0.914106,0.851735,0.790079,0.759773,0.736777
4,0.721108,0.702457,0.699058,0.623462,0.654510,0.682234


In [27]:
x_test = pd.DataFrame(np.concatenate(x_test))

x_test.columns = ['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
x_test.head()

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,-0.628100,-0.462548,-0.363481,-0.351750,-0.312934,-0.275446
1,-0.375200,-0.457493,-0.594858,-0.606624,-0.717086,-0.770898
2,-0.846592,-0.913577,-0.976689,-1.025751,-1.008726,-1.053743
3,-1.122112,-1.121161,-1.091729,-1.044890,-0.966640,-0.955708
4,-0.896795,-0.827592,-0.691637,-0.698677,-0.605324,-0.539302


In [25]:
import scipy as sp
import scipy.fftpack
concat_train['acc_t']  = concat_train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] **2)**(1/3), axis=1)
concat_train['FFT_acc_x'] = sp.fftpack.fft(np.array(concat_train['acc_x']))
concat_train['FFT_acc_y'] = sp.fftpack.fft(np.array(concat_train['acc_y']))
concat_train['FFT_acc_t'] = sp.fftpack.fft(np.array(concat_train['acc_t']))
x_test['acc_t']  = x_test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] **2)**(1/3), axis=1)
x_test['FFT_acc_x'] = sp.fftpack.fft(np.array(x_test['acc_x']))
x_test['FFT_acc_y'] = sp.fftpack.fft(np.array(x_test['acc_y']))
x_test['FFT_acc_t'] = sp.fftpack.fft(np.array(x_test['acc_t']))

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
X = concat_train
y = concat_label
X.shape, y.shape

In [96]:
# X = tf.reshape(np.array(train.iloc[:,2:]), [-1,600,6]) 
# X = np.asarray(X) 
# X.shape

In [97]:
# y = train_label['label'].values 
# y.shape

In [56]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads):
        
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation = 'relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, dff, rate = 0.1):
        
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):
    
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 maximum_position_encoding, rate = 0.1):
        
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.maximum_position_encoding = maximum_position_encoding
        self.rate = rate

#         self.pos_encoding = positional_encoding(self.maximum_position_encoding, 
#                                                 self.d_model)
#         self.embedding = tf.keras.layers.Dense(self.d_model)
        self.pos_emb = tf.keras.layers.Embedding(input_dim = self.maximum_position_encoding, 
                                                 output_dim = self.d_model)

        self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate) 
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(self.rate)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'maximum_position_encoding': self.maximum_position_encoding,
            'dropout': self.dropout,
        })
        return config

    def call(self, x, training, mask = None):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
#         x += self.pos_encoding[:, :seq_len, :]
#         x = self.embedding(x)
        positions = tf.range(start = 0, limit = seq_len, delta = 1)
        x += self.pos_emb(positions)

        x = self.dropout(x, training = training)

        for i in range(self.num_layers):

            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [123]:
from keras_multi_head import MultiHead
def attention():  

    model = Sequential()
    model.add(MultiHead([
        Conv1D(input_shape=(600,6), filters=32, kernel_size=3, padding='same', name='Conv1'),
        Conv1D(input_shape=(600,6), filters=32, kernel_size=5, padding='same', name='Conv2'),
        Conv1D(input_shape=(600,6), filters=32, kernel_size=7,padding='same', name='Conv3'),
    ], name='Multi-Head-Attention'))
    model.add(Dropout(0.4))
    model.add(MultiHead([
        Conv1D(input_shape=(600,6), filters=64, kernel_size=3, padding='same', name='Conv4'),
        Conv1D(input_shape=(600,6), filters=64, kernel_size=5, padding='same', name='Conv5'),
        Conv1D(input_shape=(600,6), filters=64, kernel_size=7,padding='same', name='Conv6'),
    ], name='Multi-Head-Attention2'))
    model.add(keras.layers.Flatten(name='Flatten'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=61, activation='softmax', name='Dense'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    return model


In [124]:
from attention import Attention
def build_model():  
    model= Sequential()
    model.add(Conv1D(64, input_shape=(600,6), kernel_size=3, padding='same',activation='relu'))
    model.add(Conv1D(64, kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(256,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(256,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    model.add(Conv1D(512,kernel_size=3,padding='same',activation='relu'))
    model.add(Conv1D(512,kernel_size=3,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.4))
    #model.add(GlobalAveragePooling1D())
    #model.add(TimeDistributed(Flatten(), name='Flatten'))
    #model.add(Bidirectional(GRU(units=32, name='Bi-GRU')))
    #model.add(GRU(64,return_sequences=True))
    #model.add(Dropout(0.4))
    
    #model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=128,
    #                                                   return_sequences=True)))
    #model.add(Flatten())
    #model.add(Attention())
    
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(61, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    return model

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kfold = StratifiedKFold(n_splits = 5, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(mskf.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1)) 
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
#     print("... Augmenting Data ...")
#     X_augmented = [] 
#     y_augmented = [] 
#     for i in tqdm(range(X_train.shape[0])): 
#         for j in range(20): 
#             shifted = np.roll(X_train[i], int(random.random() * 700)) 
#             X_augmented.append(shifted) 
#             y_augmented.append(y_train[i]) 
#     X_augmented = np.asarray(X_augmented) 
#     y_augmented = np.asarray(y_augmented)
#     X_train = np.concatenate([X_train, X_augmented]) 
#     y_train = np.concatenate([y_train, y_augmented])
    
    
    ###### feature engineering data ##### 
#     print("... Feature Engineering ...")
#     X_fourier_real = [] 
#     X_fourier_imag = [] 
#     for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
#         real_part = np.fft.fft(X_train[i]).real 
#         imag_part = np.fft.fft(X_train[i]).imag 

#         X_fourier_real.append(real_part)
#         X_fourier_imag.append(imag_part) 
    
#     X_fourier_real = np.asarray(X_fourier_real)  
#     X_fourier_imag = np.asarray(X_fourier_imag)
#     X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)   
    
    
#     X_val_fourier_real = [] 
#     X_val_fourier_imag = [] 
#     for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
#         real_part = np.fft.fft(X_val[i]).real 
#         imag_part = np.fft.fft(X_val[i]).imag 
#         X_val_fourier_real.append(real_part) 
#         X_val_fourier_imag.append(imag_part)
    
#     X_val_fourier_real = np.asarray(X_val_fourier_real) 
#     X_val_fourier_imag = np.asarray(X_val_fourier_imag)
#     X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)
        
    
    ##### train model #####  
    print("... Training ...") 
    model = attention() 
    model_path = './kfold/' + str(idx+1) + 'epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    #learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32,
              validation_data = (X_val, y_val),
              callbacks = [checkpoint, early_stopping])

C:\Users\gusals\anaconda3\envs\deep\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


... Validating on fold 1 ...
... Training ...
Epoch 1/200


In [8]:
gru1 = load_model('./kfold/1epoch_045_val_0.011.h5')
gru2 = load_model('./kfold/2epoch_040_val_0.013.h5') 
gru3 = load_model('./kfold/3epoch_041_val_0.015.h5')  
gru4 = load_model('./kfold/4epoch_041_val_0.014.h5') 
gru5 = load_model('./kfold/5epoch_066_val_0.008.h5')

In [9]:
x_test.shape

(782, 600, 6)

In [12]:
pred1 = gru1.predict(x_test) 
pred2 = gru2.predict(x_test)
pred3 = gru3.predict(x_test) 
pred4 = gru4.predict(x_test) 
pred5 = gru5.predict(x_test)

In [13]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5)/5.0

In [14]:
submission.iloc[:,1:] = pred_avg

In [15]:
submission.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,4.151162e-15,3.458583e-21,1.837691e-21,4.706263e-19,1.641891e-07,1.904628e-22,1.324257e-18,4.950554e-18,4.019481e-21,...,2.062765e-07,3.987792e-11,9.807912e-10,1.588842e-24,1.349132e-21,3.701916e-34,2.429907e-14,5.947187e-01,2.717999e-30,1.880695e-09
1,3126,4.060099e-06,8.223476e-10,1.115528e-10,2.015533e-05,8.942793e-08,1.696470e-04,5.272901e-12,6.910622e-06,6.445728e-05,...,9.043380e-07,3.344014e-09,5.665017e-11,3.160349e-09,7.622304e-11,1.712188e-10,2.005292e-03,2.268102e-12,1.559114e-11,4.736424e-04
2,3127,8.407600e-08,3.418907e-05,9.069059e-17,3.699610e-12,1.155622e-24,4.325111e-21,2.142217e-05,1.962011e-11,1.874574e-13,...,3.352589e-24,8.941564e-35,3.311255e-31,7.366345e-06,3.561864e-22,2.690297e-13,1.158918e-14,2.274473e-26,1.935776e-09,1.112074e-19
3,3128,2.711893e-04,7.834983e-11,5.505019e-12,1.183145e-08,3.976141e-11,1.825397e-04,1.207761e-14,2.449024e-05,4.845005e-07,...,2.779869e-10,6.297653e-12,4.604476e-17,1.105020e-10,5.210835e-14,7.456736e-10,1.268570e-05,2.131369e-12,1.499115e-12,8.298068e-08
4,3129,1.911167e-02,4.319781e-10,2.526147e-14,1.427498e-08,3.781315e-01,1.455914e-04,1.448823e-15,1.944142e-06,2.627512e-08,...,3.388366e-07,2.004171e-08,2.910642e-11,4.391534e-12,4.048421e-15,2.114779e-13,1.145833e-04,2.235625e-07,9.088950e-06,1.163529e-06


In [16]:
submission.to_csv('cnn_5_fold5.csv',index=False)